In [18]:
import pandas as pd
import qgrid
import matplotlib.pyplot as plt
import time
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import time
from jupyter_ui_poll import (
    ui_events, 
    with_ui_events,
    run_ui_poll_loop
)
import datetime as dt
import numpy as np
from scipy import stats
from scipy.signal import medfilt

# Loading the raw synetica data and metadata

In [8]:
synetica_data_all = pd.read_csv('synetica_data_marizu.csv')

In [9]:
synetica_meters = pd.read_excel('Synetica_meter_list.xlsx')

# Filters the synetica metadata to create separate dataframes for each consumption type and the meter IDs that measure them

In [10]:
synetica_meters_electricity = synetica_meters[(synetica_meters['Measured Units'] == 'kWh') & (synetica_meters['Meter Type'] == 'Electricity')]
synetica_meters_gas = synetica_meters[(synetica_meters['Measured Units'] == 'm3') & (synetica_meters['Meter Type'] == 'Gas')]
synetica_meters_water = synetica_meters[(synetica_meters['Measured Units'] == 'm3') & (synetica_meters['Meter Type'] == 'Water')]

# Joins the previously created dataframes to the actual data - so now each dataframe contains the readings for water, gas, electricity respectively

In [11]:
synetica_data_water = pd.merge(synetica_data_all, synetica_meters_water, left_on='name', right_on='Meter ID', how='inner')
synetica_data_gas = pd.merge(synetica_data_all, synetica_meters_gas, left_on='name', right_on='Meter ID', how='inner')
synetica_data_electricity = pd.merge(synetica_data_all, synetica_meters_electricity, left_on='name', right_on='Meter ID', how='inner')

# Heat was not included above as it has meters with different units (kwH, mWh, mWhx10), so this is processed separately. mWh, mWh x10 data is multiplied by 1000 and 10000 respectively to turn it into kWh form. Only then is all the data merged together into synetica_data_heat.

In [12]:
synetica_meters_heat = synetica_meters[(synetica_meters['Measured Units'] == 'kWh') & (synetica_meters['Meter Type'] == 'Heat')]

synetica_data_heat = pd.merge(synetica_data_all, synetica_meters_heat, left_on='name', right_on='Meter ID', how='inner')

synetica_meters_heat_mwh = synetica_meters[(synetica_meters['Measured Units'] == 'MWh') & (synetica_meters['Meter Type'] == 'Heat') & (synetica_meters['Meter ID'] != 'MC061-L04/M5') & (synetica_meters['Meter ID'] != 'MC061-L04/M6')]

synetica_data_heat2 = pd.merge(synetica_data_all, synetica_meters_heat_mwh, left_on='name', right_on='Meter ID', how='inner')

synetica_data_heat2['reading'] = synetica_data_heat2['reading']*1000

synetica_data_heat = synetica_data_heat.append(synetica_data_heat2)

synetica_meters_heat_mwh10 = synetica_meters[(synetica_meters['Measured Units'] == 'MWh x10') & (synetica_meters['Meter Type'] == 'Heat')]

synetica_data_heat3 = pd.merge(synetica_data_all, synetica_meters_heat_mwh10, left_on='name', right_on='Meter ID', how='inner')

synetica_data_heat3['reading'] = synetica_data_heat3['reading'] * 10000

synetica_data_heat = synetica_data_heat.append(synetica_data_heat3)

# Create an array listing all the consumption data dataframes. This array is iterated through in the main processing.

In [39]:
consumption_data = [synetica_data_electricity,synetica_data_gas,synetica_data_heat,synetica_data_water]


In [40]:
consumption_types = ['electricity','gas','heat','water']

# time_index creates an index of all timestamps at a 10 minutely interval between the dataset start and end. Used for reindexing below.

In [28]:
time_index = pd.date_range("2018-12-01 00:00", "2020-02-01 00:00", freq="10min")

# Main processor

# Iterates through each dataset, iterating through each meter, discarding it if there are too many nulls, and turning it into an interval meter if it is cumulative. 

# Combines the meters that belong to the same building, aggregates the 10 minutely data into 1 hourly data using pd.grouper.

# More detail in the code itself

In [42]:
combined_results = pd.DataFrame()

for idx, dataset in enumerate(consumption_data):#loop through each consumption type
    df = pd.DataFrame() # make new df that will store all meter data for each consumption type
    
    dataset['timestamp'] = pd.to_datetime(dataset['timestamp']) # turn to datetime format
    
    meter_group = dataset[['timestamp', 'Meter ID', 'reading']].groupby(['timestamp', 'Meter ID']).mean().reset_index() # Groupby meter,timestamp and take average of duplicates (only makes sure that there are no duplicates from clocks going back, see report)
    
    print(idx)
    
    
    for meter in meter_group['Meter ID'].unique(): # iterate through each meter
        meter_data = meter_group[meter_group['Meter ID'] == meter] # only use the meter data that belongs to current meter being iterated through
        meter_data = meter_data.set_index('timestamp') # set dataframe index to timestamp
        meter_data = meter_data.reindex(time_index) # reindex - so missing timestamps are filled, but reading column is left as null
        num_of_records = meter_data['reading'].shape[0] # get number of records
        num_of_null = meter_data['reading'].isna().sum() # get number of nulls (ie how many records were missing)
        proportion = num_of_null/num_of_records 
        #print('Meter: ' + meter)
        if((proportion > 0.60) or (meter == 'MC070-L01/M7') or (meter=='MC070-L01/M8')): # if more than 60% of records are null, go to next meter. two meters identified as faulty also skip at this point
            #print('too many nulls')
            continue
        meter_data['reading'] = meter_data['reading'].interpolate()
        meter_data['Meter ID'] = meter
        if ((len(meter.split('/')[1].split('R')) == 2) or (meter == 'MC065-L03/M9') or (meter == 'MC065-L03/M10')):
            #print('CUMULATIVE')
            meter_data['reading'].loc[(meter_data['reading'] < meter_data['reading'].cummax())] = np.nan
            meter_data['reading'] = meter_data['reading'].interpolate(limit=6)
            meter_data['reading'] = meter_data['reading'].diff()
            meter_data = meter_data[(meter_data['reading'] < 500) & (meter_data['reading'] >= 0)]
        meter_data['reading'].loc[meter_data['reading'] < 0] = 0
        df = df.append(meter_data)
        
    
    df = df.dropna(subset=['reading'])

    df['building'] = df['Meter ID'].str.split('-', expand=True)[0]

    df = df.reset_index()

    df['timestamp'] = df['index']

    grouped_df = df.groupby([(pd.Grouper(key='timestamp', freq='1H')), 'building']).sum().reset_index()
    

    grouped_df['type'] = consumption_types[idx]

    combined_results = combined_results.append(grouped_df)

0
1
2
3


In [ ]:
combined_results.to_csv('processed_consumption_data.csv', header=True, date_format='%Y-%m-%dT%H:%M:%SZ')